In [25]:
# import des bibliothèques utiles

from gensim.models import Word2Vec
import nltk
from gensim.models import KeyedVectors

from nltk.cluster import KMeansClusterer
import numpy as np

from sklearn import cluster
from sklearn import metrics
import re
import pandas as pd
import math
import string
import re
from MyFunctions import processSentences, meanVector, cosineDistance
import statistics
# NLP Packages
from gensim.models.phrases import Phrases, Phraser
import gensim.downloader as api
from gensim.models import Word2Vec


# process


In [26]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install fitz

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install pdfminer.six

Note: you may need to restart the kernel to use updated packages.


In [30]:
import re #Rational expressions
import PyPDF2
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage


In [31]:
page_content : str=''
with open("TousVosLivres - Bio de Madonna.pdf", "rb") as pdf_file:
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    page = read_pdf.pages[0]
    page_content += page.extractText()


In [32]:
def convert_pdf_to_txt(path):
    '''Convert pdf content from a file path to text

    :path the file path
    '''
    rsrcmgr = PDFResourceManager()
    codec = 'utf-8'
    laparams = LAParams()

    with io.StringIO() as retstr:
        with TextConverter(rsrcmgr, retstr, codec=codec,
                           laparams=laparams) as device:
            with open(path, 'rb') as fp:
                interpreter = PDFPageInterpreter(rsrcmgr, device)
                password = ""
                maxpages = 0
                caching = True
                pagenos = set()

                for page in PDFPage.get_pages(fp,
                                              pagenos,
                                              maxpages=maxpages,
                                              password=password,
                                              caching=caching,
                                              check_extractable=True):
                    interpreter.process_page(page)

                return retstr.getvalue()

if __name__ == "__main__":
    text=convert_pdf_to_txt("TousVosLivres - Bio de Madonna.pdf")

In [33]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#List of books along with their identifier on Project Gutenberg
# Feel free to expand the list and pick your favorite
books = { "TousVosLivres - Vie et almanach de B-Franklin": "TousVosLivres - Vie et almanach de B-Franklin.pdf",
          "TousVosLivres - Bio de Madonna": "TousVosLivres - Bio de Madonna.pdf",
          "TousVosLivres - bio de Steve Jobs": "TousVosLivres - bio de Steve Jobs.pdf",
        }

def bookSelection(bookTitle):
    bookId = books[bookTitle]
    print( "Loading data" )
    book = bookTitle+'.pdf'
    words = re.split('[^A-Za-z]+', convert_pdf_to_txt(book))
    words = [w for w in words if w!='']
    text_madonna=' '.join(words)
    return text_madonna

interact(bookSelection, bookTitle=books.keys() );


interactive(children=(Dropdown(description='bookTitle', options=('TousVosLivres - Vie et almanach de B-Frankli…

In [34]:
import pandas as pd
# import os
from src.pii import labs_df
import unittest
import sys

sys.path.append("..")


# print('')
# print('test_df = pd.read_csv("test_file_small_sample.csv")')
# test_df = pd.read_csv("test_file_small_sample.csv")
# print('pii_res = count_labels(test_df)')
# pii_res = count_labels(test_df)
# print('pii_res')
# print(pii_res)
# print('')
book='TousVosLivres - bio de Steve Jobs'
print('')
print(f'treatement for book: {book}')
test_df = pd.DataFrame({bookSelection(book)})
pii_res = pd.DataFrame(labs_df(test_df)).iloc[0,0]
print('pii_res')
print(pii_res)
print('')


treatement for book: TousVosLivres - bio de Steve Jobs
Loading data
pii_res
{'Person': ['mme Albert Einstein', 'mme Albert Einstein', 'mme Mais Apple', 'mme Mark Zuckerberg Voil', 'mme Product Red', 'mme Tim Cook', 'mme Laurene Ses', ' Palo Alto est', ' En Steve Jobs a', ' Joanne Carole Schieble a', ' Lorsque Steve a', ' Patty Trois a', ' Paul Jobs Palo Alto Paul est', ' Bill Gates a', ' Ronald Wayne a', ' Larry Brilliant a', ' Larry Brilliant a', ' Larry Brilliant a', ' Combi Wozniak a', ' HP Combi Wozniak a', ' Byte Shop Il est', ' Jobs Apple Computer Il est', ' National Semiconductor a', ' National Semiconductor a', ' National Semiconductor a', ' Apple II est', ' Xerox PARC Pour a', ' John Sculley a', ' NeXT Une NeXTstation a', ' NeXT Computer est', ' NeXT Cube est', ' World Wide Web a', ' George Lucas San Rafael a', ' Steve Jobs est', ' Pixar Disney a', ' Gil Amelio est', ' Gil Amelio est', ' Mac OS a', ' Mac OS a', ' Steve Jobs a', ' Steve Jobs est', ' Steve Jobs a', ' En Forbes 

In [35]:
df_madonna=pd.DataFrame(pii_res['Person'])
df_madonna['label']='Person'
df_madonna.to_csv('SteveJobs.csv',index=False)

# Training

In [36]:

def nlp_pipeline(text):
    text = text.lower() # mettre les mots en minuscule
# Retirons les caractères spéciaux :
    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\-", "", text)
    return text

In [37]:
# Import Data
reddit = pd.read_csv("Franklin_afterProcess.csv")
reddit=reddit[reddit['Truth_Value']==1]

reddit2 = pd.read_csv("madonna_afterProcess.csv")
reddit2=reddit2[reddit2['Truth_Value']==1]

reddit3 = pd.read_csv("SteveJobs_afterProcess.csv")
reddit3=reddit[reddit3['Truth_Value']==1]
sentences_raw = []


In [38]:
list(reddit['0'])[0]

'Benjamin Franklin est'

In [39]:
for i in range(len(reddit)):
    body = list(reddit['0'])[0]
    if isinstance(body, str):
        sentences_raw.extend(body.split('.'))
for i in range(len(reddit2)):
  title = list(reddit2['0'])[0]
  if isinstance(title, str):
      sentences_raw.append(title)

for i in range(len(reddit3)):
    title = list(reddit3['0'])[0]
    if isinstance(title, str):
      sentences_raw.append(title)

In [40]:

import re
import string
from gensim.models.phrases import Phrases, Phraser

'''
Clean String
'''
def cleanString(s : str):

    s = s.lower()

    # remove emojis
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    s = regrex_pattern.sub(r'',s)

    # remove non-alphabet characters
    myPunc = '!"#%&$()*+-./:;,<=>?@[\\]^_`{|}~'
    s = s.translate(s.maketrans(myPunc, ' '*len(myPunc)))
    s = s.translate(s.maketrans(string.digits, ' '*len(string.digits)))

    # clean up extra white-space
    s = re.sub('\s+',' ',s)
    return s.strip()

'''
Find Bigrams
'''
def findBigrams(corpus : list, min_count : int):

    corpusPhrased = []
    # this actually finds the bigrams
    phrases = Phrases(corpus, min_count = min_count, delimiter='_')

    for sent in corpus:
        phrased = phrases[sent]
        corpusPhrased.append(phrased)

    return corpusPhrased

'''
Process Sentences
'''
def processSentences(sentences : list, minStringSize = 5, minTokenCount = 3, splitonPeriod = False, phraseMinCount = 25, bigrams=True, trigrams=True):

    # apply string cleaning
    sentences_clean = sentences

    # lets filter out any short strings, as they likely wont hold enough information.
    sentences_clean = list(filter(lambda x: len(x) > minStringSize, sentences_clean))

    # okay so now lets go ahead and tokenize our sentences into words.
    # were going to ignore any sentences with fewer than 3 words.

    sentences_tokenized = []

    for sentence in sentences_clean:

        # for some of the larger bodies of text we are first going to split on periods to get 'real' sentences.
        if splitonPeriod:
            sents_split = sentence.split('.')

            for sent in sents_split:
                tokens = sent.split(' ')
                # were only going to care about 3 or greater.
                if len(tokens) >= minTokenCount:

                    sentences_tokenized.append(tokens)

        # dont split on period.
        else:
            tokens = sentence.split(' ')
            if len(tokens) >= minTokenCount:
                sentences_tokenized.append(tokens)


    # Time to find Bigrams and Trigrams
    if bigrams:
        sentences_tokenized = findBigrams(sentences_tokenized, phraseMinCount)
    if trigrams:
        sentences_tokenized = findBigrams(sentences_tokenized, phraseMinCount)

    return sentences_tokenized

# Now lets go ahead and apply our string processing to our raw sentences.
sentences_clean = processSentences(sentences_raw, minStringSize = 5, minTokenCount = 3, splitonPeriod = True, phraseMinCount = 25, bigrams=True, trigrams=False)

In [41]:
# import text8 corpus
text8 = api.load('text8')

# start with text8 as a base.
model = Word2Vec(text8, min_count=2, window =3, sg = 1, vector_size=100,workers=5)

# we can then continue to train on our model on the sentences we have created.
model.build_vocab(sentences_clean, update=True)
model.train(sentences_clean,total_examples=len(sentences_clean), epochs = 10)

(75, 810)

In [46]:
df= pd.DataFrame(text8)

In [48]:
df.shape

(1701, 10000)

In [49]:
import numpy as np
from numpy import ndarray, dot, array, float32 as REAL


'''
FUNCTION : meanVector(...)
INPUT :
        keyedVectors : word vectors or keyed vectors from gensim model, (model.wv)
        positive : list of words or vectors to be applied positively [default = list()]
        negative : list of words or vectors to be applied negatively [default = list()]
OUTPUT :
        averaged word vector, [type = numpy.ndarray]
DESCRIPTION :
        allows for simple averaging of positive and negative words and vectors given a gensim model's word vector library.
        NOTE: this code is pulled from gensim's word2vec repo, I just edited it to return the averaged vector.
'''

KEY_TYPES = (str, int, np.integer)

def meanVector(keyedVectors, positive=list(), negative=list()):

    # remove any words that arent in the vocabulary
    positive = list(filter(lambda x: (x in keyedVectors), positive))
    negative = list(filter(lambda x: (x in keyedVectors), negative))

    positive = [
            (item, 1.0) if isinstance(item, KEY_TYPES + (ndarray,))
            else item for item in positive
            ]
    negative = [
            (item, -1.0) if isinstance(item, KEY_TYPES + (ndarray,))
            else item for item in negative
            ]

    # compute the weighted average of all keys
    all_keys, mean = set(), []
    for key, weight in positive + negative:
        if isinstance(key, ndarray):
            mean.append(weight * key)
        else:
            mean.append(weight * keyedVectors.get_vector(key, norm=True))
            if keyedVectors.has_index_for(key):
                all_keys.add(keyedVectors.get_index(key))
        if not mean:
            raise ValueError("cannot compute similarity with no input")

    mean = matutils.unitvec(array(mean).mean(axis=0)).astype(REAL)

    return mean

'''
FUNCTION : cosineDistance(...)
'''
def cosineDistance(v1, v2):
    # matutils.unitvec scales the vectors
    return dot(matutils.unitvec(v1), matutils.unitvec(v2))

In [59]:
model.wv.similarity(w1='madonna',w2='name')

0.2616544

In [60]:
sims = []

for v in model.wv.vectors:
    sims.append(cosineDistance(v, myVec))
fig = px.histogram(pd.DataFrame({'value':sims}), x="value")
fig.add_vline(x=0.55, line_width=3, line_dash="dash", line_color="red")
fig.update_xaxes(title_text="Similarity")

fig.show()

NameError: name 'myVec' is not defined

In [ ]:
# kclusterer = KMeansClusterer(2, distance=nltk.cluster.util.cosine_distance, repeats=25)
# clusters = kclusterer.cluster(X, assign_clusters=True)
# print (clusters)

# for index, phrase in enumerate(phrases):
#     print (str(clusters[index]) + ":" + str(phrase))

# kmeans = cluster.KMeans(n_clusters=2)
# kmeans.fit(X)
# labels = kmeans.labels_
# centroids = kmeans.cluster_centers_

In [ ]:
# # On commence par utiliser notre pipeline définie plus haut

# phrases_propres = []

# for phrase in phrases:
#     phrases_propres.append(nlp_pipeline(phrase))

# # Nous devons séparer les phrases en liste de mots

# phrases_split = []

# for phrase in phrases_propres:
#     phrases_split.append(phrase.split(" "))

# # C'est là que Word2vec intervient
# X = []

# for phrase in phrases_split:
#     vec_phrase = []
#     for mot in phrase:
#         vec_phrase.append(w2v.wv[mot])
#     X.append(np.mean(vec_phrase,0))